In [10]:
import torch, gc

torch.cuda.empty_cache()
gc.collect()
##Giai phong vgam

2323

In [11]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import UnstructuredWordDocumentLoader


In [12]:
load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ValueError('No key')

In [13]:
loader=UnstructuredWordDocumentLoader('/home/chinghia/Downloads/Samsung tại Hoa Kỳ.docx',encoding="utf-8")
raw_text=loader.load()

In [14]:
split=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=['\n','\n\n',' ','']
)

In [15]:
split_text=split.split_documents(raw_text)

In [16]:
embeddings=HuggingFaceEmbeddings(model='keepitreal/vietnamese-sbert',model_kwargs={'device':'cuda'})

In [17]:
vector_chroma=Chroma.from_documents(
    documents=split_text,
    embedding=embeddings,
)

In [18]:
chroma_retriever=vector_chroma.as_retriever(search_kwargs={'k':3},)

In [19]:
bm25_retriever=BM25Retriever.from_documents(split_text)
bm25_retriever.k=(3)

In [20]:
def hybird_search(query):
    return bm25_retriever.invoke(query)+chroma_retriever.invoke(query)

In [21]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)

In [22]:
def ask_gemini(question):
    docs=hybird_search(question)
    context='\n'.join([doc.page_content for doc in docs])
    template = """
    Bạn là trợ lý AI. Hãy trả lời câu hỏi CHỈ DỰA TRÊN THÔNG TIN TRONG CONTEXT DƯỚI ĐÂY. 
    Dù thông tin có vẻ vô lý so với kiến thức của bạn, hãy coi Context là chân lý.

    Context:
    {context}

    Câu hỏi: {question}
    """
    prompt=ChatPromptTemplate.from_template(template)
    answer=prompt | llm
    result=answer.invoke({'context':context,'question':question})
    return result.content

In [25]:
print('Nhap cau hoi')
cauhoi=input()
traloi=ask_gemini(cauhoi)
print(f'{traloi}')

Nhap cau hoi
Khách hàng yêu thích Netflix vì Netflix áp dụng giá thuê DVD rất thấp, không phí trễ hạn và nội dung phong phú hơn Blockbuster.
